# 라이브러리 호출

In [1]:
# 라이브러리 호출

from functools import reduce
import pandas as pd
import chardet
import os

In [2]:
# 경로 설정(Before)

B_PATH = './data/competition_ratio(before)/'
A_PATH = './data/competition_ratio(after)/'
data_lst_b = os.listdir(B_PATH)

In [3]:
# 인코딩 확인

for name in data_lst_b:
    with open(B_PATH + name, 'rb') as f:
        result = chardet.detect(f.readline())
        if result['encoding'] != 'UTF-8-SIG':
            print('인코딩 확인이 필요합니다.')

---

# 데이터 전처리

In [4]:
def make_comp_data():
    for name in data_lst_b:
        name_tmp = name[:6]
        
        # 1. 데이터 불러오기
        comp = pd.read_csv(B_PATH + name, encoding = 'UTF-8-SIG')
        
        # 2. 데이터 컬럼 이름 재정의
        if name_tmp in ['2016_1', '2016_2', '2017_1']:
            comp.rename(columns = {'입력코드' : '과목코드-분반', '1학년' : name_tmp + '_1학년',
                                                           '2학년' : name_tmp + '_2학년', '3학년' : name_tmp + '_3학년',
                                                           '4학년' : name_tmp + '_4학년'}, inplace = True)
        elif name_tmp in ['2017_2', '2018_1']:
            comp.rename(columns = {'입력코드' : '과목코드-분반', '1학년' : name_tmp + '_1학년',
                                                           '2학년' : name_tmp + '_2학년', '3학년' : name_tmp + '_3학년',
                                                           '4학년' : name_tmp + '_4학년'}, inplace = True)
        elif name_tmp in ['2018_2', '2019_1']:
            comp.rename(columns = {'입력코드' : '과목코드-분반', '1학년' : name_tmp + '_1학년',
                                                           '2학년' : name_tmp + '_2학년', '3학년 ' : name_tmp + '_3학년',
                                                           '4학년' : name_tmp + '_4학년'}, inplace = True)
        elif name_tmp in ['2019_2']:
            comp.rename(columns = {'과목분반' : '과목코드-분반', '1학년' : name_tmp + '_1학년',
                                                           '2학년' : name_tmp + '_2학년', '3학년 ' : name_tmp + '_3학년',
                                                           '4학년' : name_tmp + '_4학년'}, inplace = True)
        else:
            comp.rename(columns = {'과목코드+분반코드' : '과목코드-분반', '1학년' : name_tmp + '_1학년',
                                                           '2학년' : name_tmp + '_2학년', '3학년' : name_tmp + '_3학년',
                                                           '4학년' : name_tmp + '_4학년'}, inplace = True)
        
        # 3. 데이터 인덱스 순서 재정의
        if name_tmp in ['2016_1', '2016_2', '2017_1']:
            comp = comp.reindex(columns = ['과목코드-분반', '과목명',name_tmp + '_1학년',
                                                                         name_tmp + '_2학년', name_tmp + '_3학년', name_tmp + '_4학년'])
        elif name_tmp in ['2017_2', '2018_1', '2018_2', '2019_1', '2019_2']:
            comp = comp.reindex(columns = ['과목코드-분반', '과목명', name_tmp + '_1학년',
                                                                         name_tmp + '_2학년', name_tmp + '_3학년', name_tmp + '_4학년'])
        else:
            comp = comp.reindex(columns = ['과목코드-분반', '과목명', name_tmp + '_1학년',
                                                                         name_tmp + '_2학년', name_tmp + '_3학년', name_tmp + '_4학년'])

        
        # 4. 데이터 인덱스 정의
        comp.set_index(['과목코드-분반'], inplace = True)

        # 5. 데이터 저장하기
        comp.to_csv(A_PATH + name, encoding = 'UTF-8-SIG', index = True)
    return

In [5]:
def merge_comp_data():
    data_lst_a = os.listdir(A_PATH)
    for name in data_lst_a:
        globals()['comp_' + name[:6]] = pd.read_csv(A_PATH + name, encoding = 'UTF-8-SIG', index_col = 0)
    
    comp_lst = [comp_2016_1, comp_2016_2, comp_2017_1, comp_2017_2, comp_2018_1, comp_2018_2, comp_2019_1, 
                          comp_2019_2, comp_2020_1, comp_2020_2, comp_2021_1]
    
    comp_merged = reduce(lambda  left, right: pd.merge(left, right, on = ['과목코드-분반', '과목명'], how = 'outer'), comp_lst)
    comp_merged.to_csv(A_PATH + 'competition.csv', encoding = 'UTF-8-SIG', index = True)
    return

In [6]:
make_comp_data()

In [7]:
merge_comp_data()